In [4]:
from dotenv import load_dotenv
import warnings

from environment.game import Game
from environment.game_state import GameState
from environment.generators.random_generator import RandomCrimeSceneMapGenerator
from llm.llama import Llama
from llm.story_generator import StoryGenerator

load_dotenv(dotenv_path="./llm-dungeon-adventures/.env")
warnings.filterwarnings("ignore", category=FutureWarning)

1. Create a room layout and a theme

In [5]:
number_of_rooms = 3
theme = "Smallville, Clark Kent, 2010"

crime_scene_map = RandomCrimeSceneMapGenerator().generate(number_of_rooms, 42)

2. Init Llama2

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = Llama()
llm_pipeline = HuggingFacePipeline(pipeline=llm.pipeline)

In [ ]:
import json
import re
from time import time

from llm.chains.killer_chain import KillerChain
from llm.chains.victim_chain import VictimChain

themes = [
    ['Prohibition Era', 'Speakeasy', 'Bootlegger'],
    ['Victorian London', 'Jack the Ripper', 'Scotland Yard'],
    ['1920s Chicago', 'Al Capone', 'Organized Crime'],
    ['Ancient Rome', 'Emperor Nero', 'Assassination Plot'],
    ['Futuristic Megacity', 'Cybernetic Hacker', 'Corporate Espionage'],
    ['Medieval Castle', 'Royal Poisoning', 'Intrigue'],
    ['Wild West', 'Outlaw Gang', 'Bank Heist'],
    ['Small Coastal Town', 'Missing Fisherman', 'Mysterious Disappearances'],
    ['Art World Heist', 'Stolen Masterpiece', 'Undercover Detective'],
    ['1940s Hollywood', 'Film Noir Detective', 'Murder Mystery'],
]

killer_chain, victim_chain = KillerChain(llm_pipeline), VictimChain(llm_pipeline)

results = []
for theme in themes:
    start_time = time()
    victim = victim_chain.create(theme)
    killer = killer_chain.create(theme, victim)
    end_time = time()
    
    results.append({
        "elapsed_time": end_time - start_time,
        "theme": theme,
        "victim": victim,
        "killer": killer
    })

In [ ]:
results

3. Create a story

In [ ]:
story = StoryGenerator(crime_scene_map.rooms, llm).create_new_story(number_of_suspects=number_of_rooms - 1, theme=theme, dummy=False)

story

4. Create an initial game state

In [ ]:
initial_game_state = GameState(crime_scene_map, story)

5. Create a game

In [ ]:
game = Game(initial_game_state)